# Commerce Challenge 2 - Baseline Submission

This notebook provides a simple baseline for **Commerce Challenge 2: Personalized Product Recommendations**.

**Goal**: Recommend top-10 SKUs for each customer (wide format)
**Metric**: NDCG@10 - Higher is better

## Instructions:
1. **Replace API credentials** in the first cell with your team's API key and name
2. **Run all cells** to generate and submit baseline predictions
3. **Check the output** for your submission score

This baseline uses a simple popularity-based recommender system.


In [ ]:
# 1. Initialize Client and Load Data

import pandas as pd
import numpy as np
from agentds import BenchmarkClient

# 🔑 REPLACE WITH YOUR CREDENTIALS
client = BenchmarkClient(
    api_key="your-api-key-here",        # Get from your team dashboard
    team_name="your-team-name-here"     # Your exact team name
)

# Load data from PVC paths
print("📂 Loading Commerce Challenge 2 data...")

# Load purchase history data
train_purchases = pd.read_csv("/home/jovyan/shared/datasets/Commerce/purchases_train.csv")
test_purchases = pd.read_csv("/home/jovyan/shared/datasets/Commerce/purchases_test.csv")

print(f"✅ Data loaded:")
print(f"   Train purchases: {train_purchases.shape}")
print(f"   Test purchases: {test_purchases.shape}")
print(f"   Train columns: {list(train_purchases.columns)}")
print(f"   Test columns: {list(test_purchases.columns)}")


In [ ]:
# 2. Frequency-Based Baseline (Personalized)

# Load products_sub to get valid SKUs for Challenge 2
products_sub = pd.read_csv("/home/jovyan/shared/datasets/Commerce/products_sub.csv")
valid_skus = set(products_sub['sku_id'].tolist())

# Filter train_purchases to only include valid SKUs
train_filtered = train_purchases[train_purchases['sku_id'].isin(valid_skus)]

print(f"📊 Filtered training data: {train_filtered.shape[0]} purchases for {len(valid_skus)} valid SKUs")

# 1. Per-customer SKU frequency table
freq_tbl = (train_filtered
            .groupby(["customer_id", "sku_id"])
            .size()
            .rename("cnt")
            .reset_index())

# 2. Global popularity for tie-breaking
glob_pop = (train_filtered["sku_id"]
            .value_counts()
            .rename_axis("sku_id")
            .rename("glob_cnt")
            .reset_index())

# Merge global popularity into freq table
freq_tbl = freq_tbl.merge(glob_pop, on="sku_id", how="left")

print(f"✅ Built frequency table: {len(freq_tbl)} customer-SKU pairs")

# Get test customers who need recommendations
test_customers = test_purchases['customer_id'].unique()
print(f"🎯 Need to recommend for {len(test_customers)} customers")

# Create personalized recommendations
recommendations = []

for customer_id in test_customers:
    # Get customer's frequency scores
    customer_freq = freq_tbl[freq_tbl['customer_id'] == customer_id].copy()
    
    if len(customer_freq) > 0:
        # Customer has purchase history - use frequency + global popularity
        customer_freq['score'] = customer_freq['cnt'] + 0.0001 * customer_freq['glob_cnt']
        recommended_skus = customer_freq.nlargest(10, 'score')['sku_id'].tolist()
    else:
        # New customer - use global popularity
        recommended_skus = glob_pop.head(10)['sku_id'].tolist()
    
    # Ensure we have exactly 10 recommendations
    if len(recommended_skus) < 10:
        # Fill remaining with other popular SKUs
        remaining_skus = [sku for sku in glob_pop['sku_id'].tolist() if sku not in recommended_skus]
        recommended_skus.extend(remaining_skus[:10-len(recommended_skus)])
    
    recommended_skus = recommended_skus[:10]
    
    # Create row: customer_id, sku_id_1, sku_id_2, ..., sku_id_10
    row = [customer_id] + recommended_skus
    recommendations.append(row)

# Create submission DataFrame in wide format
columns = ['customer_id'] + [f'sku_id_{i}' for i in range(1, 11)]
submission_df = pd.DataFrame(recommendations, columns=columns)

# Save predictions
submission_df.to_csv("commerce_challenge2_predictions.csv", index=False)
print(f"✅ Predictions saved: {submission_df.shape[0]} customers")
print(f"   Preview: {submission_df.head(3)}")
print(f"   Format: Personalized frequency-based recommendations")


In [ ]:
# 3. Submit Predictions

# Submit predictions to the competition
print("🚀 Submitting predictions...")

try:
    result = client.submit_prediction("Commerce", 2, "commerce_challenge2_predictions.csv")
    
    if result['success']:
        print("✅ Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        print(f"   📏 Metric: {result['metric_name']}")
        print(f"   ✔️  Validation: {'Passed' if result['validation_passed'] else 'Failed'}")
    else:
        print("❌ Submission failed!")
        print(f"   Error details: {result.get('details', {}).get('validation_errors', 'Unknown error')}")
        
except Exception as e:
    print(f"💥 Submission error: {e}")
    print("🔧 Check your API key and team name are correct!")

print("\n🎯 Next steps:")
print("   1. Try incorporating relevant information outside this table!")
print("   2. Move on to Commerce Challenge 3!")
